## 상대 모멘텀
- 여러개의 종목을 이용하여 투자를 전략
    1. 월초부터 월말까지의 수정 종가를 이용하여 수익율 계산
    2. 월별 수익율이 높은 n개를 선택 
    3. 해당하는 종목들을 매수하여 해장하는 종목의 신호가 없어지면 매도하는 형식으로 수익율 생성

In [ ]:
import pandas as pd 
import numpy as np 
import warnings
import os 
from glob import glob
from datetime import datetime

In [ ]:
# 경고 메시지 출력 필터 
warnings.filterwarnings('ignore')

In [ ]:
# 기준 년월 컬럼과 월별 수익율 컬럼을 생성하는 함수
# 매개변수 5개 
    # 데이터프레임 : 필수
    # ticker : 필수
    # 투자의 시작시간 : 2010-01-01 기본값
    # 투자의 종료시간 : 현재 시간 기본값
    # 기준이 되는 컬럼 : 'Adj Close' 기본값
def create_1m_rtn(
    _df, 
    _ticker, 
    _start = '2010-01-01', 
    _end = datetime.now(), 
    _col = 'Adj Close'
):
    # 복사본 생성 
    df = _df.copy()
    # 'Date'가 컬럼에 존재하면 인덱스로 변환
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    # 인덱스를 시계열 변환
    df.index = pd.to_datetime(df.index)
    # 투자의 시작시간과 종료시간으로 인덱스 필터링
    # 기준이되는 컬럼으로 컬럼 필터링 
    df = df.loc[_start : _end, [_col]]
    # 기준년월 컬럼을 생성해서 index에서 년도-월 추출하여 대입
    df['STD-YM'] = df.index.strftime('%Y-%m')
    # 월별 수익율 컬럼을 생성 
    df['1m_rtn'] = 0
    # ticker를 컬럼에 대입 
    df['CODE'] = _ticker
    # 기준년월의 유니크값을 생성
    ym_list = df['STD-YM'].unique()
    return df, ym_list



In [ ]:
aapl = pd.read_csv("data/AAPL.csv")

In [ ]:
aapl_df, ym_list = create_1m_rtn(aapl, 'AAPL')

In [ ]:
aapl_df

In [ ]:
ym_list

In [ ]:
aapl_df.loc[ym_list[1], ].iloc[-1, 0] / \
    aapl_df.loc[ym_list[1], ].iloc[0, 0]

In [ ]:
# 특정 위치의 파일의 목록을 로드 하는 기능 
os.listdir('./data')

In [ ]:
files = glob("./data/*.csv")

- 반복문을 이용 
1. data 폴더에 있는 파일들을 로드
2. create_1m_rtn() 함수에 대입 ( 데이터프레임, 파일명 )
    - price_df, ym_list 변수에 대입
3. price_df를 새로운 데이터프레임(stock_df)에 단순 행 결합  
4. 기준년월의 유니크 값을 이용하여 반복문 생성
    - 기준년월의 데이터를 이용하여 인덱스 필터를 하여 월초, 월말의 데이터를 이용해서 월별 수익율 계산
    - price_df에 월별 수익율을 대입
    - price_df에 월말의 행을 새로운 데이터프레임(month_last_df) 단순 행결합

In [ ]:
# 비어있는 데이터프레임 생성 
stock_df = pd.DataFrame()
month_last_df = pd.DataFrame()

# files를 이용해서 반복문을 생성 
for file in files:
    # file : 특정 경로와 파일명
    # print(file)
    # 경로 파일명을 나눠준다. 
    folder, name = os.path.split(file)
    # 파일명에서 이름과 확장자로 나눠준다. 
    head, tail = os.path.splitext(name)
    # head는 create_1m_rtn에서 _ticker 매개변수에 넣어줄 인자값

    # 데이터프레임 로드 
    read_df = pd.read_csv(file)
    # 함수 호출 
    price_df, ym_list = create_1m_rtn(read_df, head)
    # price_df를 stock_df에 단순 행 결합
    stock_df = pd.concat([stock_df, price_df], axis=0)

    # 두번째 반복문 생성 
    # 월별 수익율을 계산하여 대입
    for ym in ym_list:
        # ym : 기준년월
        # 월초의 가격 (매수)
        buy = price_df.loc[ym, ].iloc[0, 0]
        # 월말의 가격 (매도)
        sell = price_df.loc[ym, ].iloc[-1, 0]
        # 수익율 생성
        rtn = sell / buy
        # 수익율 대입 
        price_df.loc[ym, '1m_rtn'] = rtn
        # 월말의 데이터를 month_last_df에 단순 행 결합 
        last_df = price_df.loc[ym, ['CODE', '1m_rtn']].tail(1)
        month_last_df = pd.concat(
            [month_last_df, last_df], axis=0)
            



In [ ]:
month_last_df['CODE'].unique()

In [ ]:
# 백업 
month_rtn_df = month_last_df.copy()

In [ ]:
# 인덱스를 리셋 
month_rtn_df.reset_index(inplace=True)

In [ ]:
# 테이블을 재구조화 
month_rtn_df = month_rtn_df.pivot_table(
    index = 'Date', 
    columns = 'CODE', 
    values = '1m_rtn'
)

In [ ]:
month_rtn_df.head(1)

In [ ]:
# month_rtn_df의 데이터들을 랭크화(열의 값들은 이용)
month_rtn_df = month_rtn_df.rank(axis=1, 
    ascending=False, 
    pct=True)

In [ ]:
# 상위의 40% 종목을 선택 
# where() 함수를 사용 
    # where (조건식, 거짓일때 대입된 데이터)
month_rtn_df = \
    month_rtn_df.where( month_rtn_df <= 0.4, 0 )

In [ ]:
# 0이 아닌 데이터들은 1로 변환
month_rtn_df[month_rtn_df != 0] =1

In [ ]:
month_rtn_df.head(1)

In [ ]:
# stock_df의 code의 unique()를 변수에 저장 
stock_codes = stock_df['CODE'].unique()

In [ ]:
flag_col = month_rtn_df.loc['2010-01-29',] == 1
month_rtn_df.loc['2010-01-29' , flag_col].index

In [ ]:
# 해당 일자의 구매하려는 종목들을 딕셔너리 생성 
sig_dict = dict()

for idx in month_rtn_df.index:
    # idx : month_rtn_df의 인덱스들 (시계열)
    flag_col = month_rtn_df.loc[idx, ] == 1
    ticker_list = list(
        month_rtn_df.loc[idx, flag_col].index
    )
    # sig_dict에 추가
    sig_dict[idx] = ticker_list

In [ ]:
sig_dict

In [ ]:
# stock_df를 재구조화 
stock_c_matrix = stock_df.reset_index().pivot_table(
    index = 'Date', 
    columns = 'CODE', 
    values = stock_df.columns[0]
)

In [ ]:
stock_c_matrix.head()

In [ ]:
# 거래내역 컬럼을 추가하는 함수 생성 
def create_trade_book(_df):
    # 복사본 생성 
    df = _df.copy()
    # 컬럼의 목록을 list로 생성 
    codes = list(df.columns)
    for code in codes:
        df[f"p_{code}"] = ""
        df[f"r_{code}"] = ""
    return df

In [ ]:
book = create_trade_book(stock_c_matrix)

In [ ]:
book.head(2)

In [ ]:
# sig_dict을 이용해서 구매 대기 상태 입력 


In [ ]:
for i in sig_dict:
    print(i)
    print(sig_dict[i])
    break

In [ ]:
sig_dict.items()

In [ ]:
for i, j in sig_dict.items():
    # i = sig_dict.items()[0]
    print(i)
    print(j)
    break

In [ ]:
# sig_dict를 이용해서 구매 전 준비내역을 추가 
for date, codes in sig_dict.items():
    # date : key -> 시계열 데이터 (말일 데이터)
    # codes : value -> 종목 리스트 
    # codes 반복문 생성 
    for code in codes:
        # print(code)
        # book에서 인덱스가 date인 컬럼이 r_code인 컬럼에 준비 내역을 추가
        book.loc[date, f"p_{code}"] = f"ready_{code}" 
    # break

In [ ]:
book['p_GM'].value_counts()

In [ ]:
# 보유 내역을 추가하는 함수 생성 
def create_trading(_df, _codes):
    buy_phase = False
    df = _df.copy()
    std_ym = ""

    # _codes : 종목 리스트 -> 컬럼의 이름
    # 종목별로 순회(컬럼)하는 반복문 생성 
    for code in _codes: # 10번 반복
        # 인덱스를 기준으로 반복문 생성
        for idx in df.index:
            # 특정 종목의 포지션을 잡는다. 
            # (전행의 p_code 컬럼이 ready)이고
            # (현재 행의 p_code 컬럼이 "")
            if (df.loc[idx, f"p_{code}"] == "") & \
                (df.shift().loc[idx, f"p_{code}"] == f"ready_{code}"):
                std_ym = idx.strftime('%Y-%m')
                buy_phase = True
            # 구매 조건 : (현재 p_code가 "")이고 
            #            (index의 년도-월과 std_ym 같고)
            #            (buy_phase)
            if (df.loc[idx, f"p_{code}"] == "") & \
                (std_ym == idx.strftime('%Y-%m')) & \
                    (buy_phase):
                    df.loc[idx, f"p_{code}"] = f"buy_{code}"
            # buy_phase, std_ym 초기화
            if df.loc[idx, f"p_{code}"] == "":
                buy_phase = False
                std_ym = ""
    return df


In [ ]:
book2 = create_trading(book, stock_codes)

In [ ]:
book2['p_AMZN'].value_counts()

In [ ]:
for code in stock_codes:
    book2[f"r_{code}"] = ""

In [ ]:
book2.head(1)

In [ ]:
# 수익율 계산하는 함수 생성 
def multi_return(_df, _codes):
    # 복사본 생성 
    df = _df.copy()
    rtn = 1
    # 매수가 dict 형태로 구성 
    buy_dict = dict()
    # 매도가 dict 형태로 구성
    sell_dict = dict()

    # index를 기준으로 반복문 생성 -> 날짜별 매수, 매도 확인 
    for idx in df.index:
        # 종목별로 매수, 매도를 확인 
        for code in _codes:
            # 매수의 조건 : 2행 전(shift(2))의 p_code가 ""
            #             1행 전(shift())의 p_code가 'ready_code"
            #             현재 행의 p_code가 'buy_code'
            if (df.shift(2).loc[idx, f"p_{code}"] == "") & \
                (df.shift().loc[idx, f"p_{code}"] == f"ready_{code}") & \
                    (df.loc[idx, f"p_{code}"] == f"buy_{code}"):
                    # 매수가  -> idx 행에 code 컬럼에 존재
                    buy_dict[code] = df.loc[idx, code]
                    print(f"매수일 : {idx}, 매수 종목 : {code}, 매수가 : {df.loc[idx, code]}")
            # 매도의 조건 : 1행 전의 p_code가 buy_code
            #              현재 행의 p_code가 ""
            elif (df.shift().loc[idx, f"p_{code}"] == f"buy_{code}") & \
                (df.loc[idx, f"p_{code}"] == ""):
                # 매도가 -> idx 행에 code 컬럼에 존재 
                sell_dict[code] = df.loc[idx, code]
                # 수익율 계산
                rtn = sell_dict[code] / buy_dict[code]
                df.loc[idx, f"r_{code}"] = rtn
                print(f"매도일 : {idx}, 매도 종목 : {code}, 매도가 : {sell_dict[code]}, 수익율 : {rtn}")
            # buy_dict, sell_dict의 code 안에 매수가 매도가 초기화
            if df.loc[idx, f"p_{code}"] == "":
                buy_dict[code] = 0
                sell_dict[code] = 0
    return df



In [ ]:
rtn_book = multi_return(book2, stock_codes)

In [ ]:
rtn_book['r_AAPL'].value_counts()

In [ ]:
# 누적 수익율 계산 
def multi_acc_rtn(_df, _codes):
    # 복사본 생성 
    df = _df.copy()
    acc_rtn = 1

    # 인덱스를 기준으로 반복문 생성
    for idx in df.index:
        count = 0
        rtn = 0
        for code in _codes:
            # 수익율이 존재하는가?
            if df.loc[idx, f"r_{code}"]:
                # 존재하는 경우
                count += 1
                rtn += df.loc[idx, f"r_{code}"]
        if (rtn != 0) & (count != 0):
            acc_rtn *= rtn / count
            print(f"""누적 - 
            매도일 : {idx}, 매도 종목수 : {count}, 
            수익율 : {round( rtn / count , 2 )}""")
        df.loc[idx, 'acc_rtn'] = acc_rtn
    return df, acc_rtn


In [ ]:
# if 조건식 
    # 조건식의 타입은? bool
# if 뒤에 조건식이 타입이 bool이 아니라면? -> 강제로 형태를 bool 변환
bool( [1] )

In [101]:
multi_df, acc_rtn = multi_acc_rtn(rtn_book, stock_codes)

누적 - 
            매도일 : 2010-03-01 00:00:00, 매도 종목수 : 2, 
            수익율 : 1.01
누적 - 
            매도일 : 2010-04-01 00:00:00, 매도 종목수 : 2, 
            수익율 : 1.06
누적 - 
            매도일 : 2010-05-03 00:00:00, 매도 종목수 : 2, 
            수익율 : 1.03
누적 - 
            매도일 : 2010-06-01 00:00:00, 매도 종목수 : 2, 
            수익율 : 1.05
누적 - 
            매도일 : 2010-08-02 00:00:00, 매도 종목수 : 3, 
            수익율 : 0.98
누적 - 
            매도일 : 2010-09-01 00:00:00, 매도 종목수 : 3, 
            수익율 : 0.93
누적 - 
            매도일 : 2010-10-01 00:00:00, 매도 종목수 : 2, 
            수익율 : 1.07
누적 - 
            매도일 : 2010-11-01 00:00:00, 매도 종목수 : 1, 
            수익율 : 1.08
누적 - 
            매도일 : 2010-12-01 00:00:00, 매도 종목수 : 1, 
            수익율 : 0.97
누적 - 
            매도일 : 2011-01-03 00:00:00, 매도 종목수 : 3, 
            수익율 : 1.08
누적 - 
            매도일 : 2011-02-01 00:00:00, 매도 종목수 : 3, 
            수익율 : 1.15
누적 - 
            매도일 : 2011-03-01 00:00:00, 매도 종목수 : 3, 
            수익율 : 0.98
누적 - 
            매도일 : 2011

In [96]:
acc_rtn

9.25277276147396